# 07-3-4 HuggingFace Hub에 업로드

이 노트북은 **Hugging Face Hub**에 모델/데이터셋/Space를 업로드하는 전 과정을 Colab 기준으로 정리한 실습 파일입니다.

> ⚠️ **중요**: 개인 **Access Token**은 외부에 노출하지 마세요. 공개 저장소에 절대 커밋 금지!

## (1) Hugging Face 로그인

In [1]:
# HF_TOKEN = "hf_xxx_your_token"

from huggingface_hub import login
login(token=HF_TOKEN)  # 본인 토큰으로 교체하세요

## (2) 모델 업로드

### A. 새 모델 저장소 만들기 (CLI)

In [3]:
!hf repo create my-model --type model

usage: hf <command> [<args>]
hf: error: unrecognized arguments: --type model


### B. 로컬에 클론

In [4]:
!git clone https://huggingface.co/username/my-model
%cd my-model

Cloning into 'my-model'...
fatal: could not read Username for 'https://huggingface.co': No such device or address
[Errno 2] No such file or directory: 'my-model'
/content


### C. 모델 파일 저장
- 학습 완료된 모델 가중치(`pytorch_model.bin`, `config.json`, `tokenizer.json` 등)를 저장합니다.
- `transformers` 모델이면 `save_pretrained()`로 한 번에 저장할 수 있습니다.

In [5]:
from transformers import AutoModel, AutoTokenizer

model_id = "distilbert-base-uncased"  # 예시, 원하는 모델로 교체
model = AutoModel.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

save_dir = "."  # 현재 클론한 저장소 경로
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

('./tokenizer_config.json',
 './special_tokens_map.json',
 './vocab.txt',
 './added_tokens.json',
 './tokenizer.json')

### D. Git LFS 설정 및 푸시

In [6]:
!git lfs install
!git add .
!git commit -m "Upload my model"
!git push

Git LFS initialized.
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


## (3) 데이터셋 업로드

### A. 새 데이터셋 저장소 만들기 (CLI)

In [8]:
!hf repo create my-dataset --type dataset

usage: hf <command> [<args>]
hf: error: unrecognized arguments: --type dataset


### B. 로컬 클론

In [9]:
%cd /content
!git clone https://huggingface.co/datasets/username/my-dataset
%cd my-dataset

/content
Cloning into 'my-dataset'...
fatal: could not read Username for 'https://huggingface.co': No such device or address
[Errno 2] No such file or directory: 'my-dataset'
/content


### C. 데이터 준비 (예시 구조)
```
├── data.csv
├── README.md
└── dataset_infos.json
```
- 데이터 파일: `csv`, `json`, `parquet` 등 지원
- `README.md` 및 `dataset_infos.json` 작성 권장

In [10]:
import pandas as pd, json
pd.DataFrame({"text":["hello","world"],"label":[1,0]}).to_csv("data.csv", index=False)
open("README.md","w", encoding="utf-8").write("# My Dataset\n\n샘플 데이터셋입니다.")
open("dataset_infos.json","w", encoding="utf-8").write(json.dumps({
    "my-dataset": {
        "description": "샘플",
        "features": {"text": {"dtype": "string"}, "label": {"dtype": "int64"}}
    }
}, ensure_ascii=False, indent=2))

175

In [11]:
!git lfs install
!git add .
!git commit -m "Upload dataset"
!git push

Git LFS initialized.
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


## (4) Spaces에 업로드

### 개요
- Hub에서 Space 생성 → SDK 선택(Gradio/Streamlit) → `app.py`와 `requirements.txt` 업로드 → 자동 빌드/서비스

### Gradio 예시 (app.py)

In [12]:
%%writefile app.py
import gradio as gr

def greet(name):
    return f"안녕하세요, {name}!"

demo = gr.Interface(fn=greet, inputs="text", outputs="text")
demo.launch()

Writing app.py


### Gradio와 Streamlit 비교

| 구분 | **Gradio** | **Streamlit** |
|---|---|---|
| **목적** | 머신러닝 모델 **데모/인터페이스** 제공에 특화 | 데이터 앱·대시보드 제작에 특화 |
| **설계 철학** | “단 몇 줄로 모델을 체험하게 하자” | “데이터 과학자도 쉽게 웹 앱을 만들자” |
| **문법 스타일** | 컴포넌트 블록 배치 | `st.write`, `st.button` 등으로 UI 구성 |
| **사용 난이도** | `Interface(fn, inputs, outputs)` 한 줄 배포 | 자유도 높고 레이아웃 커스터마이징 용이 |
| **ML 친화성** | 텍스트/오디오/이미지 타입 강력 지원 | 시각화·대시보드·보고서 친화 |
| **생태계** | HF Spaces 기본 SDK | 독립 생태계 |
| **장점** | HF 모델 연계·빠른 프로토타이핑 | 대시보드/차트/필터 강점 |
| **단점** | 복잡 대시보드 한계 | HF와 기본 연동 없음 |

### 새 Space 만들기 & 로컬 업로드 (git)

In [ ]:
space_url = "https://huggingface.co/spaces/username/my-first-space"  # 본인 정보로 교체
print("Space URL:", space_url)

%cd /content
!git clone https://huggingface.co/spaces/username/my-first-space
%cd my-first-space
!cp /content/app.py .
open("requirements.txt","w").write("gradio\n")  # Streamlit이면 streamlit 추가
!git lfs install
!git add .
!git commit -m "Upload my Space app"
!git push

### 자동 배포
- push 후 1~2분 내 서버에서 빌드/실행 → Space URL에서 바로 확인 가능
- URL 형식: `https://huggingface.co/spaces/username/my-first-space`

## (5) Python API 업로드 (간단 방법)

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_file(
    path_or_fileobj="pytorch_model.bin",
    path_in_repo="pytorch_model.bin",
    repo_id="username/my-model",
    repo_type="model",
)

## (6) 주의사항 체크리스트
- 데이터셋은 `repo_type="dataset"`으로 변경
- **Git LFS**: 10MB 초과 파일은 반드시 LFS 사용
- **라이선스**: 공개 전 사용 조건 명시 (README, model card/dataset card)
- 문서화: README에 **모델/데이터셋 설명·입출력 예시·제약/한계**를 포함하세요

## (7) Gradio 기반 예제 템플릿
### 프로젝트 구조
```
my-first-space/
 ├── app.py
 └── requirements.txt
```

### app.py (감성 분석기)

In [13]:
%%writefile app.py
import gradio as gr
from transformers import pipeline

sentiment_analyzer = pipeline("sentiment-analysis")

def analyze(text):
    result = sentiment_analyzer(text)[0]
    label = result['label']
    score = round(result['score'], 3)
    return f"예측: {label} (신뢰도: {score})"

with gr.Blocks() as demo:
    gr.Markdown("## 🤗 Hugging Face Space 데모: 감성 분석기")
    with gr.Row():
        inp = gr.Textbox(label="입력 텍스트", placeholder="문장을 입력하세요...")
        out = gr.Textbox(label="결과")
    btn = gr.Button("분석 실행")
    btn.click(fn=analyze, inputs=inp, outputs=out)

demo.launch()

Overwriting app.py


### requirements.txt

In [14]:
open("requirements.txt","w").write("transformers\ntorch\ngradio\n")
print(open("requirements.txt").read())

transformers
torch
gradio

